In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests

In [2]:
world_oil_data ="iea-oil-market-report-2001-2016.csv"
worldoil_alldata = pd.read_csv(world_oil_data)
worldoil_alldata.head()

,DATE,FREQUENCY,REGION_NAME,INDICATOR_NAME,VALUE,INDICATOR_ID,REGION_ID,REGION_NOTES,REGION_REGIONID
0,2017-06,Monthly,Japan,Total products,85.7,KN.B47,JP,NaN,JP
1,2018-07,Monthly,Japan,Total products,80.1,KN.B47,JP,NaN,JP
2,2018-08,Monthly,Japan,Total products,87.0,KN.B47,JP,NaN,JP
3,2019-02,Monthly,Japan,Total products,82.4,KN.B47,JP,NaN,JP
4,2019-07,Monthly,Japan,Total products,85.3,KN.B47,JP,NaN,JP


In [3]:
worldoil_alldata.count()

DATE               24969
FREQUENCY          24969
REGION_NAME        24969
INDICATOR_NAME     24969
VALUE              24969
INDICATOR_ID       24969
REGION_ID          24969
REGION_NOTES          31
REGION_REGIONID    24969
dtype: int64

In [4]:
worldoil_alldata.dtypes

DATE                object
FREQUENCY           object
REGION_NAME         object
INDICATOR_NAME      object
VALUE              float64
INDICATOR_ID        object
REGION_ID           object
REGION_NOTES        object
REGION_REGIONID     object
dtype: object

In [5]:
# worldoil_alldata[['Year'], ['Month']] = worldoil_alldata['DATE'].str.split('-',1,expand = True)
# worldoil_alldata[['Year'], ['Month']] = world_oil_data.DATE.apply(lambda x: world_oil_data.Series(str(x).split("-")))
worldoil_alldata['year']= worldoil_alldata.DATE.str.split('-').str[0].tolist()
worldoil_alldata['month']= worldoil_alldata.DATE.str.split('-').str[1].tolist()

worldoil_alldata

,DATE,FREQUENCY,REGION_NAME,INDICATOR_NAME,VALUE,INDICATOR_ID,REGION_ID,REGION_NOTES,REGION_REGIONID,year,month
0,2017-06,Monthly,Japan,Total products,85.70,KN.B47,JP,NaN,JP,2017,06
1,2018-07,Monthly,Japan,Total products,80.10,KN.B47,JP,NaN,JP,2018,07
2,2018-08,Monthly,Japan,Total products,87.00,KN.B47,JP,NaN,JP,2018,08
3,2019-02,Monthly,Japan,Total products,82.40,KN.B47,JP,NaN,JP,2019,02
4,2019-07,Monthly,Japan,Total products,85.30,KN.B47,JP,NaN,JP,2019,07
...,...,...,...,...,...,...,...,...,...,...,...
24964,2003-07,Quarterly,Euro,Demand (mb/d),8.40,KN.B21,KN.DE1,NaN,KN.DE1,2003,07
24965,2004-04,Quarterly,Euro,Demand (mb/d),8.10,KN.B21,KN.DE1,NaN,KN.DE1,2004,04
24966,2005-07,Quarterly,Euro,Demand (mb/d),8.26,KN.B21,KN.DE1,NaN,KN.DE1,2005,07
24967,2006-04,Quarterly,Euro,Demand (mb/d),7.94,KN.B21,KN.DE1,NaN,KN.DE1,2006,04


In [6]:
df = worldoil_alldata['REGION_NOTES'].value_counts()
df = pd. DataFrame(df)
df

,REGION_NOTES
"""Total"" consist of following areas: 1. US50 2. Europe (France, Germany, Italy, Spain and UK). 3. China 4. Japan 5. India 6. Russia 7. Brazil 8. Saudi Arabia 9. Canada 10. Korea 11. Mexico 12. Iran.",31


In [7]:
del worldoil_alldata['REGION_NOTES']
del worldoil_alldata['REGION_REGIONID']
del worldoil_alldata['REGION_ID']
del worldoil_alldata['INDICATOR_ID']
del worldoil_alldata['DATE']
worldoil_alldata

,FREQUENCY,REGION_NAME,INDICATOR_NAME,VALUE,year,month
0,Monthly,Japan,Total products,85.70,2017,06
1,Monthly,Japan,Total products,80.10,2018,07
2,Monthly,Japan,Total products,87.00,2018,08
3,Monthly,Japan,Total products,82.40,2019,02
4,Monthly,Japan,Total products,85.30,2019,07
...,...,...,...,...,...,...
24964,Quarterly,Euro,Demand (mb/d),8.40,2003,07
24965,Quarterly,Euro,Demand (mb/d),8.10,2004,04
24966,Quarterly,Euro,Demand (mb/d),8.26,2005,07
24967,Quarterly,Euro,Demand (mb/d),7.94,2006,04


### Converting dtypes of year and month to integer

In [8]:
worldoil_alldata = worldoil_alldata.astype({"year":int}, errors = 'raise')
worldoil_alldata = worldoil_alldata.astype({"month":int}, errors = 'raise')
worldoil_alldata.dtypes

FREQUENCY          object
REGION_NAME        object
INDICATOR_NAME     object
VALUE             float64
year                int32
month               int32
dtype: object

### Understanding Data in columns for making DataFrame 

In [9]:
df1 = worldoil_alldata ['FREQUENCY'].value_counts()
df1 = pd.DataFrame(df1)
df1

,FREQUENCY
Monthly,15939
Quarterly,6908
Annual,2122


In [10]:
df2 = worldoil_alldata ['REGION_NAME'].value_counts()
df2 = pd.DataFrame(df2)
df2

,REGION_NAME
"Pacific, OECD",1456
"North America, OECD",1455
Canada,1270
United Kingdom,1239
United States,1177
...,...
Azerbaijan,26
Qatar,26
Kazakhstan,26
OPEC incl. Angola & Ecuador,24


### saving country / region list for DataBase users 

In [12]:
df2.to_csv("worldoildata_allcountries.csv", index= True, header = True)

### When comapred with country list used for Global emissions, the world oil database has 25 less countries but has grouped countires in regional areas like americas, latin americas etc.. 

In [13]:
df3 = worldoil_alldata ['INDICATOR_NAME'].value_counts()
df3 = pd.DataFrame(df3)
df3

,INDICATOR_NAME
Total Supply,7194
Crude oil,3472
Total,1396
Residual fuel oil,1396
Motor gasoline,1396
Other products,1348
Demand (mb/d),1025
LPG and ethane,970
Jet and kerosene,970
Naphtha,970


###  Saving Data categories for DB user

In [14]:
df3.to_csv("worldoildata_Querytype.csv", index= True, header = True)

### Understanding data catergories to finalize Dataframe loaded in SQL that would aid in further analyzing Emissions data

In [15]:
# df4 = worldoil_alldata.loc[(worldoil_alldata["INDICATOR_NAME"] == "Percentage of Summary Of Global Oil Demand"), :]
# df4 = worldoil_alldata.loc[(worldoil_alldata["INDICATOR_NAME"] == "Government"), :]
# df4 = worldoil_alldata.loc[(worldoil_alldata["INDICATOR_NAME"] == "Total Stock Stock Changes And Miscellaneous"), :]
df4 = worldoil_alldata.loc[(worldoil_alldata["INDICATOR_NAME"] == "Demand (mb/d)")
                           |(worldoil_alldata["INDICATOR_NAME"] == "Oil Demand"), :]
# df4 = worldoil_alldata.loc[(worldoil_alldata["REGION_NAME"] == "United States"), :]
df4 = pd.DataFrame(df4)
# df4 = df4.sort_values["DATE"]
df4

,FREQUENCY,REGION_NAME,INDICATOR_NAME,VALUE,year,month
44,Annual,"Pacific, OECD",Demand (mb/d),7.85,2010,1
45,Annual,"Pacific, OECD",Demand (mb/d),8.08,2012,1
198,Annual,Non-OECD,Oil Demand,48.90,2020,1
207,Annual,Non-OECD,Demand (mb/d),49.48,2016,1
208,Quarterly,Non-OECD,Demand (mb/d),49.44,2016,7
...,...,...,...,...,...,...
24964,Quarterly,Euro,Demand (mb/d),8.40,2003,7
24965,Quarterly,Euro,Demand (mb/d),8.10,2004,4
24966,Quarterly,Euro,Demand (mb/d),8.26,2005,7
24967,Quarterly,Euro,Demand (mb/d),7.94,2006,4


In [16]:
df4a = df4["REGION_NAME"].value_counts()
df4a

Latin America, Non-OECD    133
Pacific, OECD              113
North America, OECD        113
Other,Asia, Non-OECD        68
Americas, Non-OECD          63
World                       63
Former USSR                 63
Europe, OECD                63
Africa, Non-OECD            63
Europe, Non-OECD            63
Asia Oceania, OECD          63
Non-OECD                    63
Americas, OECD              63
OECD                        63
Middle East, Non-OECD       63
China                       63
Euro                        36
Iran                        31
India                       31
Brazil                      31
Japan                       31
Korea                       31
Total                       31
Europe                      31
Asia, Non-OECD              31
Saudi Arabia                31
Canada                      31
Mexico                      31
US50                        31
Russia                      31
Name: REGION_NAME, dtype: int64

### Cleaning & Organizing data for SQL - Loading org_df4 & org_worldoil_alldata in SQL. The table created in org_df4 provides the oil demand data that is related to economic health of the relative country which relates directly to emission change through the years..

In [17]:
df4 = df4.rename(columns = {"FREQUENCY": "Data_Interval", "REGION_NAME":"Country", "INDICATOR_NAME":"Data_Category", "VALUE": "million-bbls" })
org_df4 = df4[["year", "month", "Country", "Data_Interval", "Data_Category", "million-bbls"]]
org_df4.reset_index(drop = True, inplace = True)
org_df4

,year,month,Country,Data_Interval,Data_Category,million-bbls
0,2010,1,"Pacific, OECD",Annual,Demand (mb/d),7.85
1,2012,1,"Pacific, OECD",Annual,Demand (mb/d),8.08
2,2020,1,Non-OECD,Annual,Oil Demand,48.90
3,2016,1,Non-OECD,Annual,Demand (mb/d),49.48
4,2016,7,Non-OECD,Quarterly,Demand (mb/d),49.44
...,...,...,...,...,...,...
1617,2003,7,Euro,Quarterly,Demand (mb/d),8.40
1618,2004,4,Euro,Quarterly,Demand (mb/d),8.10
1619,2005,7,Euro,Quarterly,Demand (mb/d),8.26
1620,2006,4,Euro,Quarterly,Demand (mb/d),7.94


In [18]:
worldoil_alldata = worldoil_alldata.rename(columns = {"FREQUENCY": "Data_Interval", "REGION_NAME":"Country", "INDICATOR_NAME":"Data_Category", "VALUE": "million-bbls" })
org_worldoil_alldata = worldoil_alldata[["year", "month", "Country", "Data_Interval", "Data_Category", "million-bbls"]]
org_worldoil_alldata.reset_index(drop = True, inplace = True)
org_worldoil_alldata

,year,month,Country,Data_Interval,Data_Category,million-bbls
0,2017,6,Japan,Monthly,Total products,85.70
1,2018,7,Japan,Monthly,Total products,80.10
2,2018,8,Japan,Monthly,Total products,87.00
3,2019,2,Japan,Monthly,Total products,82.40
4,2019,7,Japan,Monthly,Total products,85.30
...,...,...,...,...,...,...
24964,2003,7,Euro,Quarterly,Demand (mb/d),8.40
24965,2004,4,Euro,Quarterly,Demand (mb/d),8.10
24966,2005,7,Euro,Quarterly,Demand (mb/d),8.26
24967,2006,4,Euro,Quarterly,Demand (mb/d),7.94


### Save both DataFrames to SQL

In [19]:
from config import password

In [20]:
protocol = 'postgresql'
username = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Global_Emissions'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
# connection = engine.connect()

In [22]:
engine.table_names()

['Global_Greenhouse_Emissions',
 'Global_Agri_Greenhouse_Emissions',
 'Global_Agri_Percent_Emissions',
 'Global_oil_demand',
 'Global_oil_data_iea']

In [23]:
org_worldoil_alldata.to_sql(name='Global_oil_data_iea', con=engine, if_exists='replace', index=False)

In [24]:
org_df4.to_sql(name='Global_oil_demand', con=engine, if_exists='replace', index=False)

In [25]:
pd.read_sql_query('select * from "Global_oil_data_iea"', con=engine).head()

,year,month,Country,Data_Interval,Data_Category,million-bbls
0,2017,6,Japan,Monthly,Total products,85.7
1,2018,7,Japan,Monthly,Total products,80.1
2,2018,8,Japan,Monthly,Total products,87.0
3,2019,2,Japan,Monthly,Total products,82.4
4,2019,7,Japan,Monthly,Total products,85.3


In [26]:
pd.read_sql_query('select * from "Global_oil_demand"', con=engine).head()

,year,month,Country,Data_Interval,Data_Category,million-bbls
0,2010,1,"Pacific, OECD",Annual,Demand (mb/d),7.85
1,2012,1,"Pacific, OECD",Annual,Demand (mb/d),8.08
2,2020,1,Non-OECD,Annual,Oil Demand,48.90
3,2016,1,Non-OECD,Annual,Demand (mb/d),49.48
4,2016,7,Non-OECD,Quarterly,Demand (mb/d),49.44
